<a href="https://colab.research.google.com/github/aichaoukdour/Hashing_SHA-256-FERNET_Encryption/blob/main/Generate_SQL_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keybert ctransformers[cuda]  # Use [cpu] if no GPU
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers


  Cloning https://github.com/UKPLab/sentence-transformers to /tmp/pip-req-build-nsa6_1um
  Running command git clone --filter=blob:none --quiet https://github.com/UKPLab/sentence-transformers /tmp/pip-req-build-nsa6_1um
  Resolved https://github.com/UKPLab/sentence-transformers to commit 8d73d4f0fceee03ce68df1c8e7f0c0fe843c4988
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from huggingface_hub import login

login()


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Charger Mistral
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# Créer le générateur de texte
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.7,
    do_sample=True
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cpu


In [1]:
example_prompt = """
<s>[INST]
I have the following document:
- The website mentions that it only takes a couple of days to deliver but I still have not received mine.

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else.
[/INST] deliver, days, website, mention, receive</s>"""

keyword_prompt = """
[INST]
I have the following document:
- [DOCUMENT]

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else.
[/INST]
"""

prompt = example_prompt + keyword_prompt


In [2]:
from keybert.llm import TextGeneration
from keybert import KeyLLM

# Initialiser KeyLLM
llm = TextGeneration(generator, prompt=prompt)
kw_model = KeyLLM(llm)

# Documents à traiter
documents = [
    "The website mentions that it only takes a couple of days to deliver but I still have not received mine.",
    "Meta released LLaMA model weights to the research community under a noncommercial license."
]

# Extraire les mots-clés
keywords = kw_model.extract_keywords(documents)
print(keywords)


NameError: name 'generator' is not defined

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('BAAI/bge-small-en-v1.5')
embeddings = embedding_model.encode(documents, convert_to_tensor=True)

keywords = kw_model.extract_keywords(documents, embeddings=embeddings, threshold=0.6)
